## <b><font color='darkblue'>Artifacts</font></b>
([source](https://google.github.io/adk-docs/artifacts/)) <font size='3ptx'><b>In ADK, `Artifacts` represent a crucial mechanism for managing named, versioned binary data associated either with a specific user interaction session or persistently with a user across multiple sessions</b>. They allow your agents and tools to handle data beyond simple text strings, enabling richer interactions involving files, images, audio, and other binary formats.</font>

### <b><font color='darkgreen'>What are Artifacts?</font></b>
* <b><font size='3ptx'>Definition</font></b>: An Artifact is essentially a piece of binary data (<font color='brown'>like the content of a file</font>) identified by a unique `filename` string within a specific scope (<font color='brown'>session or user</font>). Each time you save an artifact with the same filename, a new version is created.
* <b><font size='3ptx'>Representation</font></b>: Artifacts are consistently represented using the standard <b><font color='blue'>google.genai.types.Part</font></b> object. The core data is typically stored within an inline data structure of the <b><font color='blue'>Part</font></b> (accessed via `inline_data`), which itself contains:
    - <b>`data`</b>: The raw binary content as bytes.
    - <b>`mime_type`</b>: A string indicating the type of the data (e.g., `"image/png"`, `"application/pdf"`). This is essential for correctly interpreting the data later.

In [1]:
# Example of how an artifact might be represented as a types.Part
import google.genai.types as types

# Assume 'image_bytes' contains the binary data of a PNG image
image_bytes = b'\x89PNG\r\n\x1a\n...' # Placeholder for actual image bytes

image_artifact = types.Part(
    inline_data=types.Blob(
        mime_type="image/png",
        data=image_bytes
    )
)

In [2]:
# You can also use the convenience constructor:
# image_artifact_alt = types.Part.from_bytes(data=image_bytes, mime_type="image/png")

print(f"Artifact MIME Type: {image_artifact.inline_data.mime_type}")
print(f"Artifact Data (first 10 bytes): {image_artifact.inline_data.data[:10]}...")

Artifact MIME Type: image/png
Artifact Data (first 10 bytes): b'\x89PNG\r\n\x1a\n..'...


* <b><font size='3ptx'>Persistence & Management</font></b>: Artifacts are not stored directly within the agent or session state. Their storage and retrieval are managed by a dedicated <b>Artifact Service</b> (<font color='brown'>an implementation of</font> <b><font color='blue'>BaseArtifactService</font></b><font color='brown'>, defined in</font> <b><font color='blue'>google.adk.artifacts</font></b></font>). ADK provides various implementations, such as:
    - <b>An in-memory service for testing or temporary storage</b> (<font color='brown'>e.g., `InMemoryArtifactService` in Python, defined in</font> <b><font color='blue'>google.adk.artifacts.in_memory_artifact_service</font></b>).
    - <b>A service for persistent storage using Google Cloud Storage</b> (GCS) (<font color='brown'>e.g., `GcsArtifactService` in Python, defined in</font> <b><font color='blue'>google.adk.artifacts.gcs_artifact_service</font></b>). The chosen service implementation handles versioning automatically when you save data.

### <b><font color='darkgreen'>Why Use Artifacts?</font></b>
([source](https://google.github.io/adk-docs/artifacts/#why-use-artifacts)) <font size='3ptx'>While session state is suitable for storing small pieces of configuration or conversational context (<font color='brown'>like strings, numbers, booleans, or small dictionaries/lists</font>), <b>Artifacts are designed for scenarios involving binary or large data</b></font>:

1. <b><font size='3ptx'>Handling Non-Textual Data</font></b>: Easily store and retrieve images, audio clips, video snippets, PDFs, spreadsheets, or any other file format relevant to your agent's function.
2. <b><font size='3ptx'>Persisting Large Data</font></b>: Session state is generally not optimized for storing large amounts of data. Artifacts provide a dedicated mechanism for persisting larger blobs without cluttering the session state.
3. <b><font size='3ptx'>User File Management</font></b>: Provide capabilities for users to upload files (<font color='brown'>which can be saved as artifacts</font>) and retrieve or download files generated by the agent (<font color='brown'>loaded from artifacts</font>).
4. <b><font size='3ptx'>Sharing Outputs</font></b>: Enable tools or agents to generate binary outputs (<font color='brown'>like a PDF report or a generated image</font>) that can be saved via `save_artifact` and later accessed by other parts of the application or even in subsequent sessions (<font color='brown'>if using user namespacing</font>).
5. <b><font size='3ptx'>Caching Binary Data</font></b>: Store the results of computationally expensive operations that produce binary data (<font color='brown'>e.g., rendering a complex chart image</font>) as artifacts to avoid regenerating them on subsequent requests.

In essence, whenever your agent needs to work with file-like binary data that needs to be persisted, versioned, or shared, Artifacts managed by an <b><font color='blue'>ArtifactService</font></b> are the appropriate mechanism within ADK.

### <b><font color='darkgreen'>Common Use Cases</font></b>
([source](https://google.github.io/adk-docs/artifacts/#common-use-cases)) <font size='3ptx'><b>Artifacts provide a flexible way to handle binary data within your ADK applications.</b></font>

Here are some typical scenarios where they prove valuable:

* <b><font size='3ptx'>Generated Reports/Files</font></b>: A tool or agent generates a report (<font color='brown'>e.g., a PDF analysis, a CSV data export, an image chart</font>).
* <b><font size='3ptx'>Handling User Uploads</font></b>: A user uploads a file (<font color='brown'>e.g., an image for analysis, a document for summarization</font>) through a front-end interface.
* <b><font size='3ptx'>Storing Intermediate Binary Results</font></b>: An agent performs a complex multi-step process where one step generates intermediate binary data (<font color='brown'>e.g., audio synthesis, simulation results</font>).
* <b><font size='3ptx'>Persistent User Data</font></b>: Storing user-specific configuration or data that isn't a simple key-value state.
* <b><font size='3ptx'>Caching Generated Binary Content</font></b>: An agent frequently generates the same binary output based on certain inputs (<font color='brown'>e.g., a company logo image, a standard audio greeting</font>).

### <b><font color='darkgreen'>Core Concepts</font></b>
([source](https://google.github.io/adk-docs/artifacts/#core-concepts)) <font size='3ptx'><b>Understanding artifacts involves grasping a few key components: the service that manages them, the data structure used to hold them, and how they are identified and versioned.</b></font>

#### <b>Artifact Service (`BaseArtifactService`)</b>
* <b><font size='3ptx'>Role</font></b>: The central component responsible for the actual storage and retrieval logic for artifacts. It defines how and where artifacts are persisted.
* <b><font size='3ptx'>Interface</font></b>: Defined by the abstract base class <b><font color='blue'>BaseArtifactService</font></b>. Any concrete implementation must provide methods for:
    - **Save Artifact**: Stores the artifact data and returns its assigned version number.
    - **Load Artifact**: Retrieves a specific version (or the latest) of an artifact.
    - **List Artifact keys**: Lists the unique filenames of artifacts within a given scope.
    - **Delete Artifact**: Removes an artifact (<font color='brown'>and potentially all its versions, depending on implementation</font>).
    - **List versions**: Lists all available version numbers for a specific artifact filename.
* <b><font size='3ptx'>Configuration</font></b>: You provide an instance of an artifact service (<font color='brown'>e.g., `InMemoryArtifactService`, `GcsArtifactService`</font>) when initializing the `Runner`. The `Runner` then makes this service available to agents and tools via the <b><font color='blue'>InvocationContext</font></b>.

```python
from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService # Or GcsArtifactService
from google.adk.agents import LlmAgent # Any agent
from google.adk.sessions import InMemorySessionService

# Example: Configuring the Runner with an Artifact Service
my_agent = LlmAgent(name="artifact_user_agent", model="gemini-2.0-flash")
artifact_service = InMemoryArtifactService() # Choose an implementation
session_service = InMemorySessionService()

runner = Runner(
    agent=my_agent,
    app_name="my_artifact_app",
    session_service=session_service,
    artifact_service=artifact_service # Provide the service instance here
)
# Now, contexts within runs managed by this runner can use artifact methods
```

#### <b>Artifact Data</b>
* <b><font size='3ptx'>Standard Representation</font></b>: Artifact content is universally represented using the <b><font color='blue'>google.genai.types.Part</font></b> object, the same structure used for parts of LLM messages.
* <b><font size='3ptx'>Key Attribute</font></b> (`inline_data`): For artifacts, the most relevant attribute is inline_data, which is a <b><font color='blue'>google.genai.types.Blob</font></b> object containing:
    - <b>data</b> (`bytes`): The raw binary content of the artifact.
    - <b>mime_type</b> (`str`): A standard MIME type string (<font color='brown'>e.g., 'application/pdf', 'image/png', 'audio/mpeg'</font>) describing the nature of the binary data. This is crucial for correct interpretation when loading the artifact.

In [5]:
import google.genai.types as types

# Example: Creating an artifact Part from raw bytes
pdf_bytes = b'%PDF-1.4...' # Your raw PDF data
pdf_mime_type = "application/pdf"

# Using the constructor
pdf_artifact = types.Part(
    inline_data=types.Blob(data=pdf_bytes, mime_type=pdf_mime_type)
)

# Using the convenience class method (equivalent)
pdf_artifact = types.Part.from_bytes(data=pdf_bytes, mime_type=pdf_mime_type)

In [6]:
print(f"Created Python artifact with MIME type: {pdf_artifact.inline_data.mime_type}")

Created Python artifact with MIME type: application/pdf


#### <b>Filename</b>
* <b><font size='3ptx'>Identifier</font></b>: A simple string used to name and retrieve an artifact within its specific namespace.
* <b><font size='3ptx'>Uniqueness</font></b>: Filenames must be unique within their scope (<font color='brown'>either the session or the user namespace</font>).
* <b><font size='3ptx'>Best Practice</font></b>: Use descriptive names, potentially including file extensions (<font color='brown'>e.g., "monthly_report.pdf", "user_avatar.jpg"</font>), although the extension itself doesn't dictate behavior – the `mime_type` does.

#### <b>Versioning</b>
* <b><font size='3ptx'>Automatic Versioning</font></b>: The artifact service automatically handles versioning. When you call `save_artifact`, the service determines the next available version number (<font color='brown'>typically starting from 0 and incrementing</font>) for that specific filename and scope.
* <b><font size='3ptx'>Returned by `save_artifact`</font></b>: The `save_artifact` method returns the integer version number that was assigned to the newly saved artifact.
* <b><font size='3ptx'>Retrieval</font></b>:
    - `load_artifact(..., version=None)` (default): Retrieves the latest available version of the artifact.
    - `load_artifact(..., version=N)`: Retrieves the specific version `N`.
* <b><font size='3ptx'>Listing Versions</font></b>: The list_versions method (on the service, not context) can be used to find all existing version numbers for an artifact.

#### <b>Namespacing (Session vs. User)</b>
* <b><font size='3ptx'>Concept</font></b>: Artifacts can be scoped either to a specific session or more broadly to a user across all their sessions within the application. This scoping is determined by the filename format and handled internally by the <b><font color='blue'>ArtifactService</font></b>.
* <b><font size='3ptx'>Default</font></b> (`Session Scope`): If you use a plain filename like "`report.pdf`", the artifact is associated with the specific `app_name`, `user_id`, and `session_id`. It's only accessible within that exact session context.
* <b><font size='3ptx'>User Scope</font></b> (`"user:"` prefix): If you prefix the filename with `"user:"`, like `"user:profile.png"`, the artifact is associated only with the `app_name` and `user_id`. It can be accessed or updated from any session belonging to that user within the app.

```python
# Example illustrating namespace difference (conceptual)

# Session-specific artifact filename
session_report_filename = "summary.txt"

# User-specific artifact filename
user_config_filename = "user:settings.json"

# When saving 'summary.txt' via context.save_artifact,
# it's tied to the current app_name, user_id, and session_id.

# When saving 'user:settings.json' via context.save_artifact,
# the ArtifactService implementation should recognize the "user:" prefix
# and scope it to app_name and user_id, making it accessible across sessions for that user.
```

These core concepts work together to provide a flexible system for managing binary data within the ADK framework.

### <b><font color='darkgreen'>Interacting with Artifacts (via `Context` Objects)</font></b>
([source](https://google.github.io/adk-docs/artifacts/#interacting-with-artifacts-via-context-objects)) <font size='3ptx'><b>The primary way you interact with artifacts within your agent's logic (<font color='brown'>specifically within callbacks or tools</font>) is through methods provided by the `CallbackContext` and `ToolContext` objects.</b> These methods abstract away the underlying storage details managed by the <b><font color='blue'>ArtifactService</font></b>.</font>

#### <b>Prerequisite: Configuring the `ArtifactService`</b>
Before you can use any artifact methods via the context objects, you must provide an instance of a [<b>BaseArtifactService</b> implementation](https://google.github.io/adk-docs/artifacts/#available-implementations) (<font color='brown'>like [**InMemoryArtifactService**](https://google.github.io/adk-docs/artifacts/#inmemoryartifactservice) or [**GcsArtifactService**](https://google.github.io/adk-docs/artifacts/#gcsartifactservice)</font>) when initializing your `Runner`.

In Python, you provide this instance when initializing your `Runner`:

```python
from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService # Or GcsArtifactService
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService

# Your agent definition
agent = LlmAgent(name="my_agent", model="gemini-2.0-flash")

# Instantiate the desired artifact service
artifact_service = InMemoryArtifactService()

# Provide it to the Runner
runner = Runner(
    agent=agent,
    app_name="artifact_app",
    session_service=InMemorySessionService(),
    artifact_service=artifact_service # Service must be provided here
)
```

If no `artifact_service` is configured in the <b><font color='blue'>InvocationContext</font></b> (<font color='brown'>which happens if it's not passed to the `Runner`</font>), calling `save_artifact`, `load_artifact`, or `list_artifacts` on the context objects will raise a <b><font color='blue'>ValueError</font></b>.

#### <b>Accessing Methods</b>
The artifact interaction methods are available directly on instances of <b><font color='blue'>CallbackContext</font></b> (<font color='brown'>passed to agent and model callbacks</font>) and <b><font color='blue'>ToolContext</font></b> (<font color='brown'>passed to tool callbacks</font>). Remember that <b><font color='blue'>ToolContext</font></b> inherits from <b><font color='blue'>CallbackContext</font></b>.

**Save Artifact Code Example**:

```python
import google.genai.types as types
from google.adk.agents.callback_context import CallbackContext # Or ToolContext

async def save_generated_report_py(context: CallbackContext, report_bytes: bytes):
    """Saves generated PDF report bytes as an artifact."""
    report_artifact = types.Part.from_data(
        data=report_bytes,
        mime_type="application/pdf"
    )
    filename = "generated_report.pdf"

    try:
        version = await context.save_artifact(filename=filename, artifact=report_artifact)
        print(f"Successfully saved Python artifact '{filename}' as version {version}.")
        # The event generated after this callback will contain:
        # event.actions.artifact_delta == {"generated_report.pdf": version}
    except ValueError as e:
        print(f"Error saving Python artifact: {e}. Is ArtifactService configured in Runner?")
    except Exception as e:
        # Handle potential storage errors (e.g., GCS permissions)
        print(f"An unexpected error occurred during Python artifact save: {e}")

# --- Example Usage Concept (Python) ---
# async def main_py():
#   callback_context: CallbackContext = ... # obtain context
#   report_data = b'...' # Assume this holds the PDF bytes
#   await save_generated_report_py(callback_context, report_data)
```

<b>Loading Artifacts Code Example:</b>

```python
import google.genai.types as types
from google.adk.agents.callback_context import CallbackContext # Or ToolContext

async def process_latest_report_py(context: CallbackContext):
    """Loads the latest report artifact and processes its data."""
    filename = "generated_report.pdf"
    try:
        # Load the latest version
        report_artifact = await context.load_artifact(filename=filename)

        if report_artifact and report_artifact.inline_data:
            print(f"Successfully loaded latest Python artifact '{filename}'.")
            print(f"MIME Type: {report_artifact.inline_data.mime_type}")
            # Process the report_artifact.inline_data.data (bytes)
            pdf_bytes = report_artifact.inline_data.data
            print(f"Report size: {len(pdf_bytes)} bytes.")
            # ... further processing ...
        else:
            print(f"Python artifact '{filename}' not found.")

        # Example: Load a specific version (if version 0 exists)
        # specific_version_artifact = await context.load_artifact(filename=filename, version=0)
        # if specific_version_artifact:
        #     print(f"Loaded version 0 of '{filename}'.")

    except ValueError as e:
        print(f"Error loading Python artifact: {e}. Is ArtifactService configured?")
    except Exception as e:
        # Handle potential storage errors
        print(f"An unexpected error occurred during Python artifact load: {e}")

# --- Example Usage Concept (Python) ---
# async def main_py():
#   callback_context: CallbackContext = ... # obtain context
#   await process_latest_report_py(callback_context)
```

<b>Listing Artifact Filenames Code Example:</b>
```python
from google.adk.tools.tool_context import ToolContext

def list_user_files_py(tool_context: ToolContext) -> str:
    """Tool to list available artifacts for the user."""
    try:
        available_files = await tool_context.list_artifacts()
        if not available_files:
            return "You have no saved artifacts."
        else:
            # Format the list for the user/LLM
            file_list_str = "\n".join([f"- {fname}" for fname in available_files])
            return f"Here are your available Python artifacts:\n{file_list_str}"
    except ValueError as e:
        print(f"Error listing Python artifacts: {e}. Is ArtifactService configured?")
        return "Error: Could not list Python artifacts."
    except Exception as e:
        print(f"An unexpected error occurred during Python artifact list: {e}")
        return "Error: An unexpected error occurred while listing Python artifacts."

# This function would typically be wrapped in a FunctionTool
# from google.adk.tools import FunctionTool
# list_files_tool = FunctionTool(func=list_user_files_py)
```

These methods for saving, loading, and listing provide a convenient and consistent way to manage binary data persistence within ADK, whether using Python's `context` objects, regardless of the chosen backend storage implementation.

### <b><font color='darkgreen'>Available Implementations</font></b>
([source](https://google.github.io/adk-docs/artifacts/#available-implementations)) <font size='3ptx'><b>ADK provides concrete implementations of the <font color='blue'>BaseArtifactService</font> interface, offering different storage backends suitable for various development stages and deployment needs.</b> These implementations handle the details of storing, versioning, and retrieving artifact data based on the `app_name`, `user_id`, `session_id`, and `filename` (<font color='brown'>including the `user:` namespace prefix</font>).</font>

#### <b>InMemoryArtifactService</b>

* <b><font size='3ptx'>Storage Mechanism</font></b>: Uses a Python dictionary (`self.artifacts`) held in the application's memory. The dictionary keys represent the artifact path, and the values are lists of `types.Part`, where each list element is a version.
* <b><font size='3ptx'>Key Features</font></b>:
    - <b>Simplicity</b>: Requires no external setup or dependencies beyond the core ADK library.
    - <b>Speed</b>: Operations are typically very fast as they involve in-memory map/dictionary lookups and list manipulations.
    - <b>Ephemeral</b>: All stored artifacts are lost when the application process terminates. Data does not persist between application restarts.
* <b><font size='3ptx'>Use Cases</font></b>:
    - Ideal for local development and testing where persistence is not required.
    - Suitable for short-lived demonstrations or scenarios where artifact data is purely temporary within a single run of the application.
* <b><font size='3ptx'>Instantiation</font></b>:

```python
from google.adk.artifacts import InMemoryArtifactService

# Simply instantiate the class
in_memory_service_py = InMemoryArtifactService()

# Then pass it to the Runner
# runner = Runner(..., artifact_service=in_memory_service_py)
```

#### <b>GcsArtifactService</b>

* <b><font size='3ptx'>Storage Mechanism</font></b>: Leverages [**Google Cloud Storage**](https://cloud.google.com/storage?hl=en) (GCS) for persistent artifact storage. Each version of an artifact is stored as a separate object (blob) within a specified GCS bucket.
* <b><font size='3ptx'>Object Naming Convention</font></b>: It constructs GCS object names (blob names) using a hierarchical path structure.
* <b><font size='3ptx'>Key Features</font></b>:
    - <b>Persistence</b>: Artifacts stored in GCS persist across application restarts and deployments.
    - <b>Scalability</b>: Leverages the scalability and durability of Google Cloud Storage.
    - <b>Versioning</b>: Explicitly stores each version as a distinct GCS object. The `saveArtifact` method in <b><font color='blue'>GcsArtifactService</font></b>.
    - <b>Permissions Required</b>: The application environment needs appropriate credentials (<font color='brown'>e.g., Application Default Credentials</font>) and IAM permissions to read from and write to the specified GCS bucket.
* <b><font size='3ptx'>Use Cases</font></b>:
    - Production environments requiring persistent artifact storage.
    - Scenarios where artifacts need to be shared across different application instances or services (<font color='brown'>by accessing the same GCS bucket</font>).
    - Applications needing long-term storage and retrieval of user or session data.
* <b><font size='3ptx'>Instantiation</font></b>:

```python
from google.adk.artifacts import GcsArtifactService

# Specify the GCS bucket name
gcs_bucket_name_py = "your-gcs-bucket-for-adk-artifacts" # Replace with your bucket name

try:
    gcs_service_py = GcsArtifactService(bucket_name=gcs_bucket_name_py)
    print(f"Python GcsArtifactService initialized for bucket: {gcs_bucket_name_py}")
    # Ensure your environment has credentials to access this bucket.
    # e.g., via Application Default Credentials (ADC)

    # Then pass it to the Runner
    # runner = Runner(..., artifact_service=gcs_service_py)

except Exception as e:
    # Catch potential errors during GCS client initialization (e.g., auth issues)
    print(f"Error initializing Python GcsArtifactService: {e}")
    # Handle the error appropriately - maybe fall back to InMemory or raise
```

Choosing the appropriate `ArtifactService` implementation depends on your application's requirements for data persistence, scalability, and operational environment.

### <b><font color='darkgreen'>Best Practices</font></b>
([source](https://google.github.io/adk-docs/artifacts/#best-practices)) <b><font size='3ptx'>To use artifacts effectively and maintainably:</font></b>

- <b><font size='3ptx'>Choose the Right Service</font></b>: Use `InMemoryArtifactService` for rapid prototyping, testing, and scenarios where persistence isn't needed. Use `GcsArtifactService` (<font color='brown'>or implement your own `BaseArtifactService` for other backends</font>) for production environments requiring data persistence and scalability.
- <b><font size='3ptx'>Meaningful Filenames</font></b>: Use clear, descriptive filenames. Including relevant extensions (<font color='brown'>.pdf, .png, .wav</font>) helps humans understand the content, even though the `mime_type` dictates programmatic handling. Establish conventions for temporary vs. persistent artifact names.
- <b><font size='3ptx'>Specify Correct MIME Types</font></b>: Always provide an accurate `mime_type` when creating the `types.Part` for `save_artifact`. This is critical for applications or tools that later `load_artifact` to interpret the bytes data correctly. Use standard [**IANA MIME**](https://en.wikipedia.org/wiki/Media_type) types where possible.
- <b><font size='3ptx'>Understand Versioning</font></b>: Remember that `load_artifact()` without a specific version argument retrieves the latest version. If your logic depends on a specific historical version of an artifact, be sure to provide the integer version number when loading.
- <b><font size='3ptx'>Use Namespacing (`user:`) Deliberately</font></b>: Only use the `"user:"` prefix for filenames when the data truly belongs to the user and should be accessible across all their sessions. For data specific to a single conversation or session, use regular filenames without the prefix.
- <b><font size='3ptx'>Error Handling</font></b>:
    - Always check if an `artifact_service` is actually configured before calling context methods (`save_artifact`, `load_artifact`, `list_artifacts`) – they will raise a <b><font color='blue'>ValueError</font></b> if the service is `None`.
    - Check the return value of `load_artifact`, as it will be `None` if the artifact or version doesn't exist. Don't assume it always returns a `Part`.
    - Be prepared to handle exceptions from the underlying storage service, especially with `GcsArtifactService` (<font color='brown'>e.g., `google.api_core.exceptions.Forbidden` for permission issues, `NotFound` if the bucket doesn't exist, network errors</font>).
- <b><font size='3ptx'>Size Considerations</font></b>: Artifacts are suitable for typical file sizes, but <font color='darkred'>be mindful of potential costs and performance impacts with extremely large files, especially with cloud storage</font>. `InMemoryArtifactService` can consume significant memory if storing many large artifacts. Evaluate if very large data might be better handled through direct GCS links or other specialized storage solutions rather than passing entire byte arrays in-memory.
- <b><font size='3ptx'>Cleanup Strategy</font></b>: For persistent storage like `GcsArtifactService`, artifacts remain until explicitly deleted. If artifacts represent temporary data or have a limited lifespan, implement a strategy for cleanup. This might involve:
    - Using GCS lifecycle policies on the bucket.
    - Building specific tools or administrative functions that utilize the `artifact_service.delete_artifact` method (<font color='brown'>note: delete is not exposed via context objects for safety</font>).
    - Carefully managing filenames to allow pattern-based deletion if needed.